# **1. Initialize Pyspark and load dataset** 

## Install pyspark and create a SparkSession

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 33 kB/s              
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=6c006a62a85d05e83140a64171ada4c23fb504580b088d3c8a88bcdb616d4b9c
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [82]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("NetflixEDA").getOrCreate()

In [83]:
spark

## Load data

In [4]:
PATH = "/kaggle/input/netflix-subscription-price-in-different-countries/Netflix subscription fee Dec-2021.csv"

In [49]:
df_auto = spark.read.csv(PATH, inferSchema=True, header=True)

In [50]:
df_auto.printSchema()

root
 |-- Country_code: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Total Library Size: integer (nullable = true)
 |-- No. of TV Shows: integer (nullable = true)
 |-- No. of Movies: integer (nullable = true)
 |-- Cost Per Month - Basic ($): double (nullable = true)
 |-- Cost Per Month - Standard ($): double (nullable = true)
 |-- Cost Per Month - Premium ($): double (nullable = true)



In [73]:
new_names = [
    'countryCode',
    'country',
    'totalLibrarySize',
    'numberTVShows',
    'numberMovies',
    'monthlyCostBasic',
    'monthlyCostStandard',
    'monthlyCostPremium'
]

df = df.toDF(*new_names)

# **2. A glance at the data**


In [74]:
df.show(5)

+-----------+---------+----------------+-------------+------------+----------------+-------------------+------------------+
|countryCode|  country|totalLibrarySize|numberTVShows|numberMovies|monthlyCostBasic|monthlyCostStandard|monthlyCostPremium|
+-----------+---------+----------------+-------------+------------+----------------+-------------------+------------------+
|         ar|Argentina|            4760|         3154|        1606|            3.74|                6.3|              9.26|
|         au|Australia|            6114|         4050|        2064|            7.84|              12.12|             16.39|
|         at|  Austria|            5640|         3779|        1861|            9.03|              14.67|             20.32|
|         be|  Belgium|            4990|         3374|        1616|           10.16|              15.24|             20.32|
|         bo|  Bolivia|            4991|         3155|        1836|            7.99|              10.99|             13.99|
+-------

In [13]:
df.limit(5).toPandas()

,countryCode,country,totalLibrarySize,numberTVShows,numberMovies,monthlyCostBasic,monthlyCostStandard,monthlyCostPremium
0,ar,Argentina,4760,3154,1606,3.74,6.30,9.260000
1,au,Australia,6114,4050,2064,7.84,12.12,16.389999
2,at,Austria,5640,3779,1861,9.03,14.67,20.320000
3,be,Belgium,4990,3374,1616,10.16,15.24,20.320000
4,bo,Bolivia,4991,3155,1836,7.99,10.99,13.990000


In [58]:
df.describe().show()

+-------+-----------+---------+-----------------+------------------+------------------+------------------+-------------------+------------------+
|summary|countryCode|  country| totalLibrarySize|     numberTVShows|      numberMovies|  monthlyCostBasic|monthlyCostStandard|monthlyCostPremium|
+-------+-----------+---------+-----------------+------------------+------------------+------------------+-------------------+------------------+
|  count|         65|       65|               65|                65|                65|                65|                 65|                65|
|   mean|       null|     null|5314.415384615385|3518.9538461538464|1795.4615384615386|  8.36846140714792|  11.98999995451707|15.612922998575064|
| stddev|       null|     null|980.3226333124288| 723.0105555671635| 327.2797483099835|1.9378186644783801| 2.8639787613790837| 4.040672256237365|
|    min|         ar|Argentina|             2274|              1675|               373|              1.97|                3.

In [59]:
df.select("countryCode", "country").show(df.count())

+-----------+--------------+
|countryCode|       country|
+-----------+--------------+
|         ar|     Argentina|
|         au|     Australia|
|         at|       Austria|
|         be|       Belgium|
|         bo|       Bolivia|
|         br|        Brazil|
|         bg|      Bulgaria|
|         ca|        Canada|
|         cl|         Chile|
|         co|      Colombia|
|         cr|    Costa Rica|
|         hr|       Croatia|
|         cz|       Czechia|
|         dk|       Denmark|
|         ec|       Ecuador|
|         ee|       Estonia|
|         fi|       Finland|
|         fr|        France|
|         de|       Germany|
|         gi|     Gibraltar|
|         gr|        Greece|
|         gt|     Guatemala|
|         hn|      Honduras|
|         hk|     Hong Kong|
|         hu|       Hungary|
|         is|       Iceland|
|         in|         India|
|         id|     Indonesia|
|         ie|       Ireland|
|         il|        Israel|
|         it|         Italy|
|         jp| 

# **3. Data Analysis** 

## Correlations with Pyspark

__Pearson's Correlation__

The Pearson correlation coefficient is calculated as follows:

$$r_p = \frac{COV(X, Y)}{std(X) * std(Y)}$$

where COV is the covariance between the 2 variables and std the standard deviation.

In [63]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

vector_col = "corr_vars"
assembler = VectorAssembler(inputCols=no_str_df.columns, outputCol=vector_col)
df_vector = assembler.transform(no_str_df).select(vector_col)
corr_matrix = Correlation.corr(df_vector, vector_col).collect()[0][0].toArray().tolist()

/opt/conda/lib/python3.7/site-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [64]:
corr_matrix_df = spark.createDataFrame(corr_matrix, no_str_df.columns)
corr_matrix_df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    totalLibrarySize|       numberTVShows|        numberMovies|    monthlyCostBasic| monthlyCostStandard|  monthlyCostPremium|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                 1.0|  0.9711459692193751|  0.8499574080179413|-0.06287684999485411|-0.05519986931782366| -0.0715210569386201|
|  0.9711459692193751|                 1.0|  0.6997860988691629|-0.00381400892363...|0.002284882447594...|-0.01109980833468...|
|  0.8499574080179413|  0.6997860988691629|                 1.0|-0.17991345555167165|-0.17039146376748412|-0.18971058430966617|
|-0.06287684999485411|-0.00381400892363...|-0.17991345555167165|                 1.0|   0.898534825160934|  0.8209140310212804|
|-0.05519986931782366|0.002284882447594...|-0.17039146376748412|   0.898534825160934|                 1.

In [65]:
plot_corr = corr_matrix_df.toPandas()
plot_corr.index = corr_matrix_df.columns
plot_corr.style.background_gradient(cmap='Blues')

,totalLibrarySize,numberTVShows,numberMovies,monthlyCostBasic,monthlyCostStandard,monthlyCostPremium
totalLibrarySize,1.000000,0.971146,0.849957,-0.062877,-0.055200,-0.071521
numberTVShows,0.971146,1.000000,0.699786,-0.003814,0.002285,-0.011100
numberMovies,0.849957,0.699786,1.000000,-0.179913,-0.170391,-0.189711
monthlyCostBasic,-0.062877,-0.003814,-0.179913,1.000000,0.898535,0.820914
monthlyCostStandard,-0.055200,0.002285,-0.170391,0.898535,1.000000,0.978768
monthlyCostPremium,-0.071521,-0.011100,-0.189711,0.820914,0.978768,1.000000


## Countries with most expensive Netflix subscriptions

In [66]:
from pyspark.sql.functions import avg

cols = [col('monthlyCostBasic'), col('monthlyCostStandard'), col('monthlyCostPremium')]
avgColsFunc = sum(x for x in cols) / len(cols)

In [67]:
dfWithAvg = df.withColumn('AverageSubsCost', avgColsFunc)
dfWithAvg.show(5)

+-----------+---------+----------------+-------------+------------+----------------+-------------------+------------------+------------------+
|countryCode|  country|totalLibrarySize|numberTVShows|numberMovies|monthlyCostBasic|monthlyCostStandard|monthlyCostPremium|   AverageSubsCost|
+-----------+---------+----------------+-------------+------------+----------------+-------------------+------------------+------------------+
|         ar|Argentina|            4760|         3154|        1606|            3.74|                6.3|              9.26| 6.433333079020183|
|         au|Australia|            6114|         4050|        2064|            7.84|              12.12|             16.39|12.116666158040365|
|         at|  Austria|            5640|         3779|        1861|            9.03|              14.67|             20.32|14.673333485921225|
|         be|  Belgium|            4990|         3374|        1616|           10.16|              15.24|             20.32|15.240000406901041|

In [68]:
dfWithAvg.select("country", "AverageSubsCost") \
    .orderBy(col("AverageSubsCost").desc()).limit(10).show()

+-------------+------------------+
|      country|   AverageSubsCost|
+-------------+------------------+
|Liechtenstein|20.099999745686848|
|  Switzerland|20.099999745686848|
|      Denmark|15.546666463216146|
|      Belgium|15.240000406901041|
|       France|15.240000406901041|
|       Israel|15.050000508626303|
|       Sweden|14.933333079020182|
|    Gibraltar|14.673333485921225|
|      Austria|14.673333485921225|
|        Italy|14.673333485921225|
+-------------+------------------+



In [45]:
dfWithAvgRel = dfWithAvg.withColumn('AverageSubsCostPerTitle', dfWithAvg.AverageSubsCost / dfWithAvg.totalLibrarySize)
dfWithAvgRel.show(5)

+-----------+---------+----------------+-------------+------------+----------------+-------------------+------------------+------------------+-----------------------+
|countryCode|  country|totalLibrarySize|numberTVShows|numberMovies|monthlyCostBasic|monthlyCostStandard|monthlyCostPremium|   AverageSubsCost|AverageSubsCostPerTitle|
+-----------+---------+----------------+-------------+------------+----------------+-------------------+------------------+------------------+-----------------------+
|         ar|Argentina|            4760|         3154|        1606|            3.74|                6.3|              9.26| 6.433333079020183|   0.001351540562819366|
|         au|Australia|            6114|         4050|        2064|            7.84|              12.12|             16.39|12.116666158040365|   0.001981790343153478|
|         at|  Austria|            5640|         3779|        1861|            9.03|              14.67|             20.32|14.673333485921225|   0.002601654873390288

In [69]:
dfWithAvgRel.select("country", "AverageSubsCostPerTitle") \
    .orderBy(col("AverageSubsCostPerTitle").desc()).limit(10).show()

+-------------+-----------------------+
|      country|AverageSubsCostPerTitle|
+-------------+-----------------------+
|Liechtenstein|   0.006594488105540305|
|   San Marino|   0.006352092418147716|
|      Croatia|   0.004963353944642562|
|  Switzerland|   0.003650562975969...|
|       Sweden|   0.003424291006425...|
|      Denmark|   0.003410852668542...|
|      Finland|   0.003348166234315...|
|      Belgium|   0.003054108297976...|
|       Norway|   0.002934334556121287|
|      Moldova|   0.002866819118648003|
+-------------+-----------------------+



## Countries with cheapest Netflix subscriptions


In [70]:
dfWithAvg.select("country", "AverageSubsCost") \
    .orderBy(col("AverageSubsCost").asc()).limit(10).show()

+------------+-----------------+
|     country|  AverageSubsCost|
+------------+-----------------+
|      Turkey|2.996666590372721|
|       India|5.950000127156575|
|   Argentina|6.433333079020183|
|    Colombia|7.033333460489909|
|      Brazil|7.226666768391927|
|     Ukraine|8.463333129882812|
|        Peru| 8.56000010172526|
| Philippines|9.140000025431315|
|South Africa|9.630000432332357|
|       Chile| 9.90666643778483|
+------------+-----------------+



In [71]:
dfWithAvgRel.select("country", "AverageSubsCostPerTitle") \
    .orderBy(col("AverageSubsCostPerTitle").asc()).limit(10).show()

+-----------+-----------------------+
|    country|AverageSubsCostPerTitle|
+-----------+-----------------------+
|     Turkey|   6.459725351094463E-4|
|      India|   0.001018312532458767|
|  Argentina|   0.001351540562819366|
|   Colombia|   0.001409203257962314|
|Philippines|   0.001436655143890...|
|     Brazil|   0.001453472801366035|
|    Hungary|   0.001555781574090...|
|    Czechia|   0.001568600738143487|
|    Ukraine|   0.001586081920892581|
|   Slovakia|   0.001604359185517...|
+-----------+-----------------------+

